In [23]:
from src.schwefel import SchwefelProblem
from time import time
import gpax
import numpyro

In [38]:


def schwefel_1d(x):

    return 418.9829  - x * np.sin(np.sqrt(np.abs(x)))

def schwefel_nd(args):
    output = 0
    
    for dim in range(args):
        output += schwefel_1d(args[dim])

def add_gaussian_noise(signal, noise_level):

    return signal + np.random.normal(0, noise_level, 1)[0]

def schwefel_1d_with_noise(x, noise_level = 0.01):
    # Calculate the Schwefel function value

    schwefel_value = schwefel_1d(x)

    # Add Gaussian noise to the Schwefel function value

    noisy_schwefel_value = add_gaussian_noise(schwefel_value, noise_level)

    return noisy_schwefel_value

def schwefel_nd_with_noise(args, noise_level = 0.01):
    # Calculate the Schwefel function value

    schwefel_value = schwefel_nd(args)

    # Add Gaussian noise to the Schwefel function value

    noisy_schwefel_value = add_gaussian_noise(schwefel_value, noise_level)

    return noisy_schwefel_value




In [39]:

def create_data(seed, n_init, noise_level):

    np.random.seed(seed)
    X_bounds = np.array([-500,  500])
    X = np.random.uniform(X_bounds[0], X_bounds[1], size=( n_init,))
    X = np.append(X, X_bounds)
    X = np.sort(X)
    y = schwefel_1d_with_noise(X, noise_level = noise_level)

    X_unmeasured = np.linspace(X_bounds[0], X_bounds[1], 200)
    ground_truth = schwefel_1d_with_noise(X_unmeasured, noise_level = 0)
    
    return X, y, X_unmeasured, ground_truth


In [40]:
noise_prior = lambda: numpyro.sample("noise", numpyro.distributions.HalfNormal(0.01))

In [41]:
def gp_kernel_prior():
    length = numpyro.sample("k_length", numpyro.distributions.Uniform(0.1, 2)) #0.1 2
    scale = numpyro.sample("k_scale", numpyro.distributions.LogNormal(0, 1))
    # the hyperparameters are returned as dictionary
    return {"k_length": length, "k_scale": scale}

In [42]:
def step(X_measured, y_measured, X_unmeasured):
    # Get random number generator keys for training and prediction
    rng_key1, rng_key2 = gpax.utils.get_keys()
    # Initialize GP model
    gp_model = gpax.ExactGP(1, kernel='Matern', kernel_prior=gp_kernel_prior, noise_prior=noise_prior)
    #gpax.ExactGP(1, kernel='Matern')
    # Run HMC to obtain posterior samples for the GP model parameters
    gp_model.fit(rng_key1, X_measured, y_measured)
    # Get predictions (we don't need this step for optimization - only for visualization purposes)
    y_pred, y_sampled = gp_model.predict(rng_key2, X_unmeasured, noiseless=True, n= 10)

    # Compute acquisition function

    #Upper confidence bound
    #obj = gpax.acquisition.UCB(rng_key2, gp_model, X_unmeasured, beta= 10, maximize=False, noiseless=True)

    #Expected improvement
    obj = gpax.acquisition.EI(rng_key2, gp_model, X_unmeasured, xi=0.01, maximize=False, n=10, noiseless=True) #xi = 0.01

    # pure uncertainty-based
    #obj = gpax.acquisition.UE(rng_key2, gp_model, X_unmeasured, noiseless=True)

    return obj, (y_pred, y_sampled)

In [43]:
def run_gp(num_steps, X, y, X_unmeasured, ground_truth, schwefel_1d_with_noise, noise_level ):
    for e in range(num_steps):
        print("\nStep {}/{}".format(e+1, num_steps))
        # Compute acquisition function
        acq, (y_pred, y_sampled) = step(X, y, X_unmeasured)
        # Get the next point to evaluate
        idx = acq.argmin()
        next_point = X_unmeasured[idx:idx+1]

        # Measure the point
        next_point_value = schwefel_1d_with_noise(next_point, noise_level)
        #Note that in experiment this is the step when you go to the lab. Strong insentive to learn how to make convergence faster!

        # Update measured data
        X = np.append(X, X_unmeasured[idx:idx+1])
        y = np.append(y, next_point_value)

#         # Plot observed points, mean prediction, and acqusition function
#         lower_b = y_pred - y_sampled.std(axis=(0,1))
#         upper_b = y_pred + y_sampled.std(axis=(0,1))
#         fig, (ax1, ax2) = plt.subplots(1, 2, dpi=100, figsize=(14, 5.5))
#         ax1.scatter(X[:-1], y[:-1], marker='x', c='k', label="Observations", s=64)
#         ax1.fill_between(X_unmeasured, lower_b, upper_b, color='r', alpha=0.3, label="Model uncertainty")
#         ax2.plot(X_unmeasured, acq, lw=2, c='orangered', label='Acquisition function')
#         ax2.scatter(X_unmeasured[idx], acq[idx], s=90, c='orangered', label='Next point to measure')
#         for ax in fig.axes:
#             ax.plot(X_unmeasured, y_pred, lw=2, c='b', label='Posterior mean')
#             ax.plot(X_unmeasured, ground_truth, label='Ground truth')
#             #ax.set_ylim(3.0, 8)
#             ax.set_xlabel("$X$", fontsize=16)
#             ax.set_ylabel("$y$", fontsize=16)
#             ax.legend(loc='best', fontsize=10)
#         plt.show()
    # Quantifying prediction accuracy
    mse = np.mean((y_pred - ground_truth)**2)
    average_uncertainty = np.mean(y_sampled.std(axis=(0,1)))    
    return mse, average_uncertainty

# run experiments

In [46]:
import itertools
import pandas as pd

def grid_search(seeds, n_inits, noise_levels):
    results = []  # Initialize an empty list to store results

    # Iterate over all combinations of seeds, n_inits, and noise_levels
    for seed, n_init, noise_level in itertools.product(seeds, n_inits, noise_levels):
        print(f"Seed: {seed}, n_init: {n_init}, Noise Level: {noise_level}")
        
        # Create data for the current combination
        X, y, X_unmeasured, ground_truth = create_data(seed, n_init, noise_level)
        
        # Run Gaussian Process (GP) optimization/modeling for the current combination
        mse, average_uncertainty = run_gp(5, X, y, X_unmeasured, ground_truth, schwefel_1d_with_noise, noise_level)
        
        # Collect the results
        results.append({
            "Seed": seed,
            "n_init": n_init,
            "Noise_Level": noise_level,
            "MSE": mse,
            "Average_Uncertainty": average_uncertainty
        })

    # Convert the results to a pandas DataFrame for easy analysis and reporting
    results_df = pd.DataFrame(results)
    
    return results_df




In [48]:
# Define your parameter space
seeds = [1, 2, 3]
n_inits = [10, 15, 20]
noise_levels = [0, 0.01, 0.1, 0.5]

# Assuming create_data and run_gp functions are defined elsewhere

# Run the grid search
results_df = grid_search(seeds, n_inits, noise_levels)

# Display the results DataFrame
print(results_df)

Seed: 1, n_init: 10, Noise Level: 0

Step 1/5


sample: 100%|█| 4000/4000 [00:18<00:00, 219.94it/s, 3 steps of



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  k_length      0.68      0.00      0.68      0.68      0.68      1.26      1.00
   k_scale    453.36      0.00    453.36    453.36    453.36       nan       nan
     noise      0.01      0.00      0.01      0.01      0.01      0.50      1.00


Step 2/5


/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:111: FutureWarning: `noise_prior` is deprecated and will be removed in a future version. Please use `noise_prior_dist` instead, which accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.HalfNormal(scale=0.1)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:119: UserWarning: `kernel_prior` will remain available for complex priors. However, for modifying only the lengthscales, it is recommended to use `lengthscale_prior_dist` instead. `lengthscale_prior_dist` accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.Gamma(2, 5)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
sample: 100%|█| 4000/4000 [00:21<00:00, 184.16it/s, 6 steps of



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  k_length      0.68      0.00      0.68      0.68      0.68      0.50      1.00
   k_scale     43.01      0.00     43.01     43.00     43.01      6.19      1.21
     noise      0.01      0.00      0.01      0.01      0.01      0.50      1.00


Step 3/5


...


                mean       std    median      5.0%     95.0%     n_eff     r_hat
  k_length      0.69      0.00      0.69      0.69      0.69      2.88      2.15
   k_scale     65.58      0.00     65.58     65.58     65.58      0.50      1.00
     noise      0.01      0.00      0.01      0.01      0.01      0.50      1.00


Step 4/5


/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:111: FutureWarning: `noise_prior` is deprecated and will be removed in a future version. Please use `noise_prior_dist` instead, which accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.HalfNormal(scale=0.1)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:119: UserWarning: `kernel_prior` will remain available for complex priors. However, for modifying only the lengthscales, it is recommended to use `lengthscale_prior_dist` instead. `lengthscale_prior_dist` accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.Gamma(2, 5)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
sample: 100%|█| 4000/4000 [01:12<00:00, 55.18it/s, 1023 steps 



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  k_length      0.70      0.00      0.70      0.70      0.70      3.08      2.58
   k_scale    158.34      0.00    158.34    158.34    158.34       nan       nan
     noise      0.01      0.00      0.01      0.01      0.01      0.50      1.00


Step 5/5


/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:111: FutureWarning: `noise_prior` is deprecated and will be removed in a future version. Please use `noise_prior_dist` instead, which accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.HalfNormal(scale=0.1)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
/lustre/isaac/scratch/upratius/projects/gpax/gpax/models/gp.py:119: UserWarning: `kernel_prior` will remain available for complex priors. However, for modifying only the lengthscales, it is recommended to use `lengthscale_prior_dist` instead. `lengthscale_prior_dist` accepts an instance of a numpyro.distributions Distribution object, e.g., `dist.Gamma(2, 5)`, rather than a function that calls `numpyro.sample`.
  warnings.warn(
sample: 100%|█| 4000/4000 [01:17<00:00, 51.64it/s, 1023 steps 



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  k_length      0.67      0.00      0.67      0.67      0.67      0.50      1.00
   k_scale    437.95      0.00    437.95    437.95    437.95      0.50      1.00
     noise      0.01      0.00      0.01      0.01      0.01      0.50      1.00

    Seed  n_init  Noise_Level        MSE Average_Uncertainty
0      1      10         0.00  206665.11            4.705776
1      1      10         0.01  206703.03           1.1732969
2      1      10         0.10  206704.39           1.4820279
3      1      10         0.50  206695.95           1.1728493
4      1      15         0.00  206429.17            4.312067
5      1      15         0.01  206703.86                 nan
6      1      15         0.10  206194.23           41.119843
7      1      15         0.50  206438.97                 nan
8      1      20         0.00   204810.6                 nan
9      1      20         0.01  204980.75           3.6358469
10 

In [ ]:
results_df.to_csv('grid_search_results.csv', index=False)

In [49]:
results_df

,Seed,n_init,Noise_Level,MSE,Average_Uncertainty
0,1,10,0.00,206665.11,4.705776
1,1,10,0.01,206703.03,1.1732969
2,1,10,0.10,206704.39,1.4820279
3,1,10,0.50,206695.95,1.1728493
4,1,15,0.00,206429.17,4.312067
5,1,15,0.01,206703.86,nan
6,1,15,0.10,206194.23,41.119843
7,1,15,0.50,206438.97,nan
8,1,20,0.00,204810.6,nan
9,1,20,0.01,204980.75,3.6358469
